<a href="https://colab.research.google.com/github/nuriddinovN/practice_nlp/blob/main/stt_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers torch torchaudio pydub accelerate
!apt update &> /dev/null
!apt install -y ffmpeg &> /dev/null


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.6 MB/s eta 0:00:00


In [2]:
import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from pydub import AudioSegment
import tempfile
from google.colab import files
import os

print("📦 Dependencies installed!")

📦 Dependencies installed!


In [3]:
def load_model():
    models = ["jamshidahmadov/whisper-uz", "mirodil/whisper-small-uzbek", "openai/whisper-medium"]

    for model_name in models:
        try:
            print(f"Loading {model_name}...")
            processor = WhisperProcessor.from_pretrained(model_name)
            model = WhisperForConditionalGeneration.from_pretrained(model_name)
            print(f"✅ Loaded: {model_name}")
            return processor, model, model_name
        except:
            continue
    raise Exception("No model could be loaded!")

processor, model, model_name = load_model()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def transcribe_file(file_path):
    """
    Transcribe a WAV file to text

    Args:
        file_path (str): Path to the WAV file

    Returns:
        str: Transcribed text
    """

    # Check if file exists
    if not os.path.exists(file_path):
        return f"❌ File not found: {file_path}"

    print(f"📁 File: {file_path}")

    try:
        # Load audio
        audio = AudioSegment.from_file(file_path)

        # Convert to mono 16kHz
        if audio.channels > 1:
            audio = audio.set_channels(1)
        if audio.frame_rate != 16000:
            audio = audio.set_frame_rate(16000)

        print(f"⏱️ Duration: {len(audio)/1000:.1f} seconds")

        # Process with Whisper
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmp:
            audio.export(tmp.name, format="wav")
            waveform, _ = torchaudio.load(tmp.name)

            input_features = processor(
                waveform.squeeze().numpy(),
                sampling_rate=16000,
                return_tensors="pt"
            ).input_features.to(device)

            print("🤖 Transcribing...")

            with torch.no_grad():
                if "openai" in model_name:
                    generated_ids = model.generate(input_features, language="uz", task="transcribe")
                else:
                    generated_ids = model.generate(input_features)

            transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

            return transcription.strip()

    except Exception as e:
        return f"❌ Error: {str(e)}"


Loading jamshidahmadov/whisper-uz...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

✅ Loaded: jamshidahmadov/whisper-uz


In [7]:
def show_files():
    """Show current directory and WAV files"""
    current_dir = os.getcwd()
    print(f"📂 Current directory: {current_dir}")

    # List all files
    files = os.listdir(current_dir)
    wav_files = [f for f in files if f.lower().endswith(('.wav', '.mp3', '.m4a', '.flac'))]

    print(f"\n🎵 Audio files found:")
    if wav_files:
        for i, file in enumerate(wav_files, 1):
            print(f"  {i}. {file}")
    else:
        print("  No audio files found in current directory")

    print(f"\n📋 All files:")
    for file in files[:10]:  # Show first 10 files
        print(f"  - {file}")
    if len(files) > 10:
        print(f"  ... and {len(files) - 10} more files")

# Show available files
print("\n" + "="*50)
print("🎤 UZBEK SPEECH-TO-TEXT")
print("="*50)
print(f"Model: {model_name}")
print(f"Device: {device}")
print("="*50)

show_files()

print(f"\n💡 USAGE:")
print(f"transcribe_file('your_file.wav')")
print(f"\nExample:")
print(f"result = transcribe_file('audio.wav')")
print(f"print(result)")

# Example function call (uncomment and modify the path)
# result = transcribe_file('your_audio_file.wav')
# print(f"\n📝 RESULT:\n{result}")

def transcribe(path):
    """Shorthand function"""
    result = transcribe_file(path)
    print(f"\n📝 TRANSCRIPTION:")
    print("-" * 50)
    print(result)
    print("-" * 50)
    return result


🎤 UZBEK SPEECH-TO-TEXT
Model: jamshidahmadov/whisper-uz
Device: cpu
📂 Current directory: /content

🎵 Audio files found:
  1. s_1623.wav

📋 All files:
  - .config
  - s_1623.wav
  - sample_data

💡 USAGE:
transcribe_file('your_file.wav')

Example:
result = transcribe_file('audio.wav')
print(result)


In [8]:
# See what files are available
show_files()

# Test with your file (replace with actual filename)
transcribe('/content/s_1623.wav')

# Or get result as variable
text = transcribe_file('/content/s_1623.wav')

print(f"Transcribed text: {text}")

📂 Current directory: /content

🎵 Audio files found:
  1. s_1623.wav

📋 All files:
  - .config
  - s_1623.wav
  - sample_data
📁 File: /content/s_1623.wav
⏱️ Duration: 7.6 seconds


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
The attention mask is not set and cannot be inferred from input because pad token 

🤖 Transcribing...


A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> to see related `.generate()` flags.



📝 TRANSCRIPTION:
--------------------------------------------------
Sug'urta puli sug'urta hodisasi natijasida ko'rilgan zalar qiymatini sug'urtalovchi tomonidan qoplashning eng yuqori miqdoridagi summa.
--------------------------------------------------
📁 File: /content/s_1623.wav
⏱️ Duration: 7.6 seconds
🤖 Transcribing...
Transcribed text: Sug'urta puli sug'urta hodisasi natijasida ko'rilgan zalar qiymatini sug'urtalovchi tomonidan qoplashning eng yuqori miqdoridagi summa.


In [10]:
text = transcribe_file('/content/test1.wav')

print(f"Transcribed text: {text}")

📁 File: /content/test1.wav
⏱️ Duration: 4.5 seconds
🤖 Transcribing...
Transcribed text: Salom bu test versiya bari ikki uch.


In [11]:
text = transcribe_file('/content/cv_sample_2048.wav')

print(f"Transcribed text: {text}")

📁 File: /content/cv_sample_2048.wav
⏱️ Duration: 3.6 seconds
🤖 Transcribing...
Transcribed text: Siz uyushmada yosh ijodkorlar bilan shug'ullanasiz.


In [12]:
text = transcribe_file('/content/test2.wav')

print(f"Transcribed text: {text}")

📁 File: /content/test2.wav
⏱️ Duration: 5.4 seconds
🤖 Transcribing...
Transcribed text: Tasvirsiya ikki, model aniqligini tekshiramiz, mening ismin bo'lmuhammad.


In [13]:
text = transcribe_file('/content/calling.wav')

print(f"Transcribed text: {text}")

📁 File: /content/calling.wav
⏱️ Duration: 19.6 seconds
🤖 Transcribing...
Transcribed text: Ali. Qalq. A muhammad shugalama futbol. E'g'ol. Ha, futbol ham haydovchigangan. Ha, reja budjet fuq. Mayhattor.


In [14]:
text = transcribe_file('/content/calling2.wav')

print(f"Transcribed text: {text}")

📁 File: /content/calling2.wav
⏱️ Duration: 25.0 seconds
🤖 Transcribing...
Transcribed text: Avo! Aloh! Aloh! Asomalika bu Nurmiqaman bo'ladilarmi? Nurmiqaman, huquqtaman, ifonal Admishini hadib bo'laman Admishintinda Nurmiqaman, bizagik, Telegramda, masjub olganim, deyapti, menga gazetiz.
